In [1]:
# !pip install catboost
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# !pip install sweetviz
import sweetviz as sv
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.font_manager as fm
rc('font',family= 'malgun gothic')
rc('axes', unicode_minus = False)

import math
%matplotlib inline
import seaborn as sns
# !pip install -U klib
import klib
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import VotingRegressor

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm, tqdm_notebook
import platform
from itertools import combinations
from scipy.stats.mstats import gmean


import datetime
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import PCA
import optuna

C:\Users\inuya\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Loading Feature

In [2]:
# 피쳐 불러오기
num_features_train = pd.read_csv(os.path.abspath("../input")+'/seo_num_features_train.csv',encoding = 'utf-8')
num_features_test = pd.read_csv(os.path.abspath("../input")+'/seo_num_features_test.csv',encoding = 'utf-8')
onehot_features_train = pd.read_csv(os.path.abspath("../input")+'/seo_onehot_features_train.csv',encoding = 'utf-8')
onehot_features_test = pd.read_csv(os.path.abspath("../input")+'/seo_onehot_features_test.csv',encoding = 'utf-8')
w2v_features_train = pd.read_csv(os.path.abspath("../input")+'/seo_w2v_features_train.csv',encoding = 'utf-8')
w2v_features_test = pd.read_csv(os.path.abspath("../input")+'/seo_w2v_features_test.csv',encoding = 'utf-8')
target = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding = 'cp949').group

In [3]:
num_features_train.shape

(21587, 1566)

In [4]:
onehot_features_train.shape

(21587, 2314)

In [5]:
w2v_features_test.shape

(14380, 1950)

In [6]:
# custid 분리
id_train = num_features_train.custid.unique()
id_test = num_features_test.custid.unique()
num_features_train = num_features_train.drop(columns = 'custid')
num_features_test = num_features_test.drop(columns = 'custid')

In [7]:
# scaling 적용
scaler = StandardScaler()
num_features_scaler_train = scaler.fit_transform(num_features_train)
num_features_scaler_test = scaler.fit(num_features_train).transform(num_features_test)

### Merge All Featuers

In [8]:
all_features_scaler_train =  pd.concat([pd.DataFrame(num_features_scaler_train), onehot_features_train, w2v_features_train], axis = 1)
all_features_scaler_test =  pd.concat([pd.DataFrame(num_features_scaler_test), onehot_features_test, w2v_features_test], axis = 1)

In [9]:
all_features_train = all_features_scaler_train
all_features_test = all_features_scaler_test

### Feature Selection

In [10]:
clf = LGBMClassifier(random_state = 0)     

In [11]:
from sklearn.feature_selection import SelectFromModel

In [12]:
all_features_train_fs = all_features_train
all_features_test_fs = all_features_test

In [13]:
smf = SelectFromModel(clf, threshold='3.0*mean')
smf.fit(all_features_train_fs, target)

X_new = smf.transform(all_features_train_fs)
X_te_new = smf.transform(all_features_test_fs)

feature_selection_idx = smf.get_support()
feature_selection_name = all_features_train_fs.columns[feature_selection_idx]

In [14]:
all_features_train_fs.columns[feature_selection_idx]

Index([                   7,                   19,                   33,
                         35,                   38,                   43,
                         47,                   71,                   78,
                         91,
       ...
       'customer_info_v284', 'customer_info_v285', 'customer_info_v287',
       'customer_info_v288', 'customer_info_v290', 'customer_info_v291',
       'customer_info_v294', 'customer_info_v296', 'customer_info_v297',
       'customer_info_v298'],
      dtype='object', length=540)

In [15]:
# csv파일로 저장.
X_new = pd.DataFrame(X_new)
X_te_new = pd.DataFrame(X_te_new)

X_new.columns = feature_selection_name
X_te_new.columns = feature_selection_name

X_new.to_csv('seo_X_new_selected.csv',encoding = 'utf-8',index = False)
X_te_new.to_csv('seo_X_te_new_selected.csv',encoding = 'utf-8',index = False)